<a href="https://colab.research.google.com/github/Aditya-titus/Machine-learning/blob/main/Neural_network_cw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Neural Network**

### Importing functions

In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import *
from google.colab import drive, files

### Reading files and extracting data

In [32]:
# Reading file

drive.mount('/content/gdrive')
data = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/dataset1.csv')

# Reading dataset and separating input features (X) and target labels (y)
X = data.iloc[:, :-1]  # Input features
y = data.iloc[:, -1]   # Target labels (1 for hitting, 0 for missing)


# Finding scaling parameters using mean and standard deviation and storing a numpy array
mean = X.mean()
std = X.std()

# COnverting to a 2 x 6 numpy array
scale_array = np.vstack((mean, std))

# Save the features array using np.savetxt
np.savetxt('/content/gdrive/MyDrive/Colab Notebooks/AT2720-1.txt', scale_array)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### Defining training sets and applying scaling to data

In [33]:
# Load scaling parameters from the saved text file
scale_array = np.loadtxt('/content/gdrive/MyDrive/Colab Notebooks/AT2720-1.txt')

# Applying scaling to the dataset
X_array = X.values

print(scale_array)

X_scaled = (X_array - scale_array[0])/scale_array[1]
y_array = y.values

[[2.99790586e-01 3.10500234e-01 1.61670426e+02 1.88135000e+01
  2.84024574e+02 4.27726820e+00]
 [9.83924728e-02 9.97600486e-02 8.83980565e+01 4.82435021e+00
  1.37933777e+02 2.21237117e+00]]


### Defining training and testing dataset

In [34]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_array, test_size=0.2, random_state=42)

# Convert labels to categorical
y_train_binary = to_categorical(y_train)
y_test_binary = to_categorical(y_test)

### Code to find the Optimal Neural Network to this dataset and the time taken

In [30]:
# Define a function to create and compile the model
def create_model(units, activation, input_dim):
    model = Sequential()
    model.add(Dense(units=units, activation='relu', input_dim=input_dim))
    model.add(Dense(units=units/2, activation=activation))
    model.add(Dense(units=2, activation='sigmoid'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


# Define hyperparameters
units_list = [6,12] # nodes
activations = ['relu', 'softmax', 'tanh', 'softplus', 'sigmoid']

# Store results (accuracy, model)
results = []

# Iterate through hyperparameters
for units in units_list:
    for activation in activations:
        # Calculating time to implement the model
        st = time.time()
        # Create and compile the model
        model = create_model(units, activation, X_train.shape[1])

        # Train the model
        model.fit(X_train, y_train_binary, epochs=50, batch_size=32, verbose=0)

        et = time.time()

        # Evaluate the model
        _, accuracy = model.evaluate(X_test, y_test_binary, verbose=0)

        # Store the results
        results.append((units, activation, accuracy, model))

        # Print the accuracy for each configuration
        print(f"Units: {units}, Activation: {activation}, Accuracy: {accuracy}, Time: {et-st} ms")

# Find the best configuration and accuracy
best_config = max(results, key=lambda x: x[2])
best_units, best_activation, best_accuracy, best_model = best_config

# Print the best configuration and accuracy
print(f"Best Configuration: Units={best_units}, Activation={best_activation}, Accuracy={best_accuracy}")

Units: 6, Activation: relu, Accuracy: 0.9150000214576721, Time: 6.011020183563232 ms
Units: 6, Activation: softmax, Accuracy: 0.9150000214576721, Time: 11.382629156112671 ms
Units: 6, Activation: tanh, Accuracy: 0.9200000166893005, Time: 5.9556801319122314 ms
Units: 6, Activation: softplus, Accuracy: 0.9125000238418579, Time: 11.369725704193115 ms
Units: 6, Activation: sigmoid, Accuracy: 0.8999999761581421, Time: 6.739586114883423 ms
Units: 12, Activation: relu, Accuracy: 0.9325000047683716, Time: 6.201934337615967 ms
Units: 12, Activation: softmax, Accuracy: 0.9175000190734863, Time: 5.902160167694092 ms
Units: 12, Activation: tanh, Accuracy: 0.9275000095367432, Time: 6.268803358078003 ms
Units: 12, Activation: softplus, Accuracy: 0.9175000190734863, Time: 5.238459825515747 ms
Units: 12, Activation: sigmoid, Accuracy: 0.9175000190734863, Time: 11.030265808105469 ms
Best Configuration: Units=12, Activation=relu, Accuracy=0.9325000047683716


### Implementing the most optimal solution